In [8]:
# Remember to pip install pytest

import pandas as pd
from utils import read_cin_data
import pytest

(
    ChildCharacteristics,
    ChildIdentifiers,
    ChildProtectionPlans, # No data in here
    CINdetails,
    CINplanDates, # No data in here
    Header,
    Reviews, # No data in here
    Section47, # No data in here
) = read_cin_data()

We'll try rules as simply doing slices before we try doing them as functions, and before we write tests for them

Rules to try:
- rule 4180: <GenderCurrent> (N00097) must be present and valid
- rule 4220: <Ethnicity> (N00177) must be present and a valid code
- rule 100: <ReferenceDate> (N00603) must be present and must equal 2022-03-31
- rule 1520: Each pupil <UPN> (N00001) must be unique across all pupils in the extract
- rule 8510: Each <LAchildID> (N00097) must be unique across all children within the same LA return. 
- rule 1540: If <UPN> (N00001) present Characters 5-12 of <UPN> must be numeric

In [11]:
# Lets do 4180 as an example.
# For the cin validator, we find failing rows and return the index locations of those rows as a list

valid_gender_codes = ["1", "2", "0", "9"]

df = ChildIdentifiers

failing_rows = df[~df['GenderCurrent'].isin(valid_gender_codes)].index

failing_rows


Index([9, 21, 32, 45], dtype='int64')

Lets rewrite these as fucntions so we can pop them inside pytests and so it's more relevant to the validator code.

In [16]:
def rule_4180(df):
    valid_gender_codes = ["1", "2", "0", "9"]

    failing_rows = df[~df['GenderCurrent'].isin(valid_gender_codes)].index

    return list(failing_rows)


def test_rule_4180():

    ChildIdentifiers = pd.DataFrame(
        {'GenderCurrent':["1", 
                          "2", 
                          "0",
                          "9",
                          "4", # 4, fail
                          1, #5, fail
                          "just a word" # 6, fail
                          ]}
    )

    result = rule_4180(ChildIdentifiers)

    assert result == [4, 5, 6]




SyntaxError: invalid syntax (3783209921.py, line 24)